# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234016950937                   -0.50    7.0         
  2   -7.250084804072       -1.79       -1.40    1.0   5.10ms
  3   -7.250925604869       -3.08       -1.87    2.0   5.79ms
  4   -7.250927334332       -5.76       -1.85    2.0   6.25ms
  5   -7.251332787838       -3.39       -2.67    1.0   5.17ms
  6   -7.251337987820       -5.28       -3.15    1.0   5.27ms
  7   -7.251338588644       -6.22       -3.38    2.0   6.34ms
  8   -7.251338788855       -6.70       -4.05    1.0   5.39ms
  9   -7.251338796384       -8.12       -4.33    3.0   7.29ms
 10   -7.251338798543       -8.67       -5.11    1.0   5.46ms
 11   -7.251338798685       -9.85       -5.49    3.0   7.04ms
 12   -7.251338798702      -10.78       -5.73    3.0   7.74ms
 13   -7.251338798704      -11.72       -6.02    1.0   5.72ms
 14   -7.251338798704      -12.11       -6.61    2.0   6.60ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.060518860031454985
[ Info: Arnoldi iteration step 2: normres = 0.592046761570299
[ Info: Arnoldi iteration step 3: normres = 0.7728487434855612
[ Info: Arnoldi iteration step 4: normres = 0.21296909153927399
[ Info: Arnoldi iteration step 5: normres = 0.6113239084011638
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.16e-02, 7.22e-02, 5.49e-01, 2.55e-01, 3.46e-02)
[ Info: Arnoldi iteration step 6: normres = 0.24109654781712395
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.46e-03, 1.63e-01, 1.23e-01, 1.01e-01, 3.74e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10377490265930715
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.28e-04, 1.58e-02, 9.65e-03, 7.21e-02, 6.82e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09523720655433511
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.55e